In this project we propose to create an item-item recommendation system that will assist users in selecting the next title to watch based on their input of a specific title. We will adopt this approach instead of building user profiles, so the imputed title will be used in lieu of a user profile, however user profile data is available that we may incorporate into our project later, time allowing.

We have begun setting up our item-item system, but we have not progressed very far.

As you can see, we have calculated the most popular and high rated titles, as well as averages of the like. This is very important for recomendations! The better rated and ranked an item, the more likely it is to be recommended. It also is important to think about which items are popular in general and which items are specifically 

In [1]:
import numpy as np
import pandas as pd

ratings_df = pd.read_csv("./input/ratings_small.csv", dtype = 'float')
ratings_df.head()

movies_df = pd.read_csv("./input/movies_metadata.csv")
movies_df.head()

movies_df = movies_df[movies_df['id'].astype(str).str.isdigit()] # clean (Some movies have not integers as id)
movies_df.id = movies_df.id.astype(np.int64)

ratings_df = pd.merge(ratings_df,movies_df[['title','id']],left_on='movieId',right_on='id')
ratings_df.head()

ratings_df.drop(['timestamp','id'],axis=1,inplace=True)

titles = ratings_df["title"].unique()
titlesRate = []
titlesVote = []

for t in range(len(titles)):
    temp = ratings_df[(ratings_df["title"] == titles[t])]
    v = temp['rating'].mean()
    titlesVote.append(temp['rating'].count())
    titlesRate.append(v)

data = {'Title':  titles,
        'Rating': titlesRate,
        'Votes': titlesVote
       }
Leaderboard = pd.DataFrame (data, columns = ['Title','Rating','Votes'])


C:\Users\Kathleen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# Highly rated movies
print("Top rated movies")
Leaderboard = Leaderboard.sort_values(by=['Rating','Votes'], ascending = False).reset_index(drop = True)
print(Leaderboard.head(10))

print("Most viewed movies")
# Popular
Leaderboard = Leaderboard.sort_values(by=['Votes','Rating'], ascending = False).reset_index(drop = True)
print(Leaderboard.head(10))

Top rated movies
                      Title  Rating  Votes
0           The Celebration     5.0      3
1   The Night of the Hunter     5.0      3
2                 Ninotchka     5.0      2
3                 Silentium     5.0      2
4  Gentlemen Prefer Blondes     5.0      2
5                 Long Pigs     5.0      2
6              Nick of Time     5.0      1
7        The Crazy Stranger     5.0      1
8            The Inquisitor     5.0      1
9      Murder in Three Acts     5.0      1
Most viewed movies
                                Title    Rating  Votes
0  Terminator 3: Rise of the Machines  4.256173    324
1            The Million Dollar Hotel  4.487138    311
2                             Solaris  4.134426    305
3                        The 39 Steps  4.221649    291
4                     Monsoon Wedding  3.706204    274
5                  Once Were Warriors  4.303279    244
6                   Three Colors: Red  3.945175    228
7                     Men in Black II  4.256696    